In [2]:
%load_ext autoreload
%autoreload 2 
from fibsem import utils, milling, acquire
from fibsem.microscope import FibsemMicroscope
from fibsem.structures import FibsemPatternSettings, FibsemPattern, FibsemImage, BeamType
import matplotlib
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
microscope, settings = utils.setup_session(config_path= r"C:\Users\lnae0002\Desktop\fibsem\fibsem\config")


2023-06-28 15:37:53,225 — root — INFO — connect_to_microscope:2241 — Microscope client connecting to [localhost:8300]
2023-06-28 15:37:53,229 — root — INFO — connect_to_microscope:2243 — Microscope client connected to [localhost:8300]
2023-06-28 15:37:54,098 — root — INFO — connect_to_microscope:2249 — Microscope client connected to model S8252G with serial number 42 and software version TESCAN Essence Version 1.2.3.0, build 6115.
2023-06-28 15:37:54,100 — root — INFO — setup_session:229 — Finished setup for session: demo_2023-06-28-03-37-53PM


In [5]:

rectangle = FibsemPatternSettings(
    pattern = FibsemPattern.Rectangle,
    width = 10e-6,
    height = 10e-6,
    rotation = 0,
    centre_x = 0,
    centre_y = 0,
    depth = 1e-6,
)
#milling.draw_bitmap(microscope, rectangle, "C:\\Users\\lnae0002\\Desktop\\autolamella_test\\2\\000001\\Empty_ref.tif")
settings.image.beam_type = BeamType.ION
img = acquire.new_image(microscope, settings.image)


2023-06-28 15:37:56,250 — root — INFO — autocontrast:2532 — Running autocontrast on ION.
2023-06-28 15:37:59,381 — root — INFO — acquire_image:2261 — acquiring new ION image.


In [8]:
milling.milling_protocol(
    microscope,
    settings.milling,
    patterns= [rectangle],
    drift_correction= True,
    image_settings = settings.image,
    ref_image = img,
)

spacing: 1.0
2023-06-28 15:38:49,322 — root — INFO — draw_rectangle:3373 — Scan direction TopToBottom not supported. Using Flyback instead.
2023-06-28 15:38:49,323 — root — INFO — draw_rectangle:3374 — Supported scan directions are: Flyback, RLE, SpiralInsideOut, SpiralOutsideIn, ZigZag
2023-06-28 15:38:49,325 — root — INFO — run_milling_drift_corrected:3274 — running ion beam milling now...
2023-06-28 15:38:54,718 — numexpr.utils — INFO — _init_num_threads:160 — NumExpr defaulting to 8 threads.
 |██████████████████████████████----------------------------------------------------------------------| 31% 
2023-06-28 15:38:55,435 — root — INFO — run_milling_drift_corrected:3297 — Drift correction in progress...
2023-06-28 15:38:55,436 — root — INFO — autocontrast:2532 — Running autocontrast on ION.
2023-06-28 15:38:58,576 — root — INFO — acquire_image:2261 — acquiring new ION image.
2023-06-28 15:39:02,079 — root — INFO — beam_shift:2582 — ION shifting by (0.0, 0.0)
2023-06-28 15:39:03,084

Exception: InvalidRequest (-1)

In [ ]:
# microscope.connection.DrawBeam.UnloadLayer()
# layer = microscope.connection.DrawBeam.Layer("Layer1")
# print(layer)
microscope.connection.Detector.GetConfiguration(0)

In [ ]:
key = "detector_type"
value = "MD"

microscope.set(key, value, BeamType.ELECTRON)
eb = acquire.new_image(microscope, settings.image)
settings.image.beam_type = BeamType.ION
microscope.set(key, value, BeamType.ION)
ib = acquire.new_image(microscope, settings.image)
fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(eb.data, cmap="gray")
ax[0].set_title("Electron Image 01")
ax[1].imshow(ib.data, cmap="gray")
ax[1].set_title("Ion Image 01")
plt.show()